In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
image = cv2.imread('./road_test.jpg')


In [13]:
lane_img = np.copy(image)

def canny(img):
    gray = cv2.cvtColor(lane_img,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(blur,50,150)
    return canny
def display_lines(img,lines):
    line_img = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(line_img,(x1,y1),(x2,y2),(255,0,0),10)
    return line_img
#             print([x1,y1,x2,y2])


def region_of_interst(image):
    height = image.shape[0]
    polygons = np.array([
        [(200,height),(1100,height),(550,250)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask,polygons,255)
    masked_img = cv2.bitwise_and(image,mask)
    return masked_img
    
canny = canny(lane_img)
lane_img = np.copy(image) 
cropped_img = region_of_interst(canny)
lines = cv2.HoughLinesP(cropped_img,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
# plt.imshow(canny)
line_img = display_lines(lane_img,lines)
combo_img = cv2.addWeighted(lane_img,0.8, line_img,1,1)
cv2.imshow('road', combo_img)
cv2.waitKey(0)



-1